In [1]:
!unzip CEDAR.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import glob

#read one folder
files = glob.glob('BHSig260-Hindi/1/*.tif')

In [3]:
import glob

#read one folder
files = glob.glob('CEDAR/1/*.png')

In [4]:
files

['CEDAR/1\\forgeries_1_1.png',
 'CEDAR/1\\forgeries_1_10.png',
 'CEDAR/1\\forgeries_1_11.png',
 'CEDAR/1\\forgeries_1_12.png',
 'CEDAR/1\\forgeries_1_13.png',
 'CEDAR/1\\forgeries_1_14.png',
 'CEDAR/1\\forgeries_1_15.png',
 'CEDAR/1\\forgeries_1_16.png',
 'CEDAR/1\\forgeries_1_17.png',
 'CEDAR/1\\forgeries_1_18.png',
 'CEDAR/1\\forgeries_1_19.png',
 'CEDAR/1\\forgeries_1_2.png',
 'CEDAR/1\\forgeries_1_20.png',
 'CEDAR/1\\forgeries_1_21.png',
 'CEDAR/1\\forgeries_1_22.png',
 'CEDAR/1\\forgeries_1_23.png',
 'CEDAR/1\\forgeries_1_24.png',
 'CEDAR/1\\forgeries_1_3.png',
 'CEDAR/1\\forgeries_1_4.png',
 'CEDAR/1\\forgeries_1_5.png',
 'CEDAR/1\\forgeries_1_6.png',
 'CEDAR/1\\forgeries_1_7.png',
 'CEDAR/1\\forgeries_1_8.png',
 'CEDAR/1\\forgeries_1_9.png',
 'CEDAR/1\\original_1_1.png',
 'CEDAR/1\\original_1_10.png',
 'CEDAR/1\\original_1_11.png',
 'CEDAR/1\\original_1_12.png',
 'CEDAR/1\\original_1_13.png',
 'CEDAR/1\\original_1_14.png',
 'CEDAR/1\\original_1_15.png',
 'CEDAR/1\\original_1_16.

In [5]:
from itertools import combinations

combinations_list = list(combinations(files, 2))

# Print each combination
print(combinations_list)

[('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_10.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_11.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_12.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_13.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_14.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_15.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_16.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_17.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_18.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_19.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_2.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_20.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_21.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_22.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_23.png'), ('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_24

In [6]:
#one value of combination
x,y=combinations_list[0]

In [7]:
label1=0 if "F" in  x.split("\\")[-1].split(".")[0] else 1
label2=0 if "F" in  y.split("\\")[-1].split(".")[0] else 1
label=label1*label2

In [8]:
x,y

('CEDAR/1\\forgeries_1_1.png', 'CEDAR/1\\forgeries_1_10.png')

In [9]:
label

1

# general function

In [10]:
import pandas as pd
import glob
from itertools import combinations

def files_to_csv(file_name, n, file_type):
    image1 = []
    image2 = []
    label = []
    labels1=[]
    labels2=[]
    
    for i in range(1, n+1):
        files = glob.glob(f'{file_name}\\{i}\\*.{file_type}')
        pairs = list(combinations(files, 2))
        
        for combo in pairs:
            label1 = 0 if "F" in  combo[0].split("\\")[-1].split(".")[0]  else 1
            label2 = 0 if "F" in combo[1].split("\\")[-1].split(".")[0]  else 1
            label.append(label1 * label2)
            image1.append(combo[0])
            labels1.append(label1)
            labels2.append(label2)
            image2.append(combo[1])
    
    # Create the DataFrame
    df = pd.DataFrame(list(zip(image1, image2, labels1,labels2,label)), columns=['Testing_Signature', 'Original_Signature','Label1','Label2', 'Ouput'])
    #remove all the rows where label2 is 0
    df=df[df.Label2!=0]
    df.to_csv(f"{file_name}_extracted.csv",index=False)

In [11]:
files_to_csv("BHSig260-Hindi",160,"tif")

In [12]:
df=pd.read_csv("BHSig260-Hindi_extracted.csv")

In [42]:
import pandas as pd
import glob
from itertools import combinations

def cedar_files_to_csv(n, file_type):
    image1 = []
    image2 = []
    label = []
    labels1=[]
    labels2=[]
    
    for i in range(1, n+1):
        files = glob.glob(f'CEDAR\\{i}\\*.{file_type}')
        pairs = list(combinations(files, 2))
        
        for combo in pairs:
            # print(combo[0].split("\\")[-1][0])
            label1 = 0 if "f" ==  combo[0].split("\\")[-1][0]  else 1
            label2 = 0 if "f" == combo[1].split("\\")[-1][0]  else 1
            labels1.append(label1)
            labels2.append(label2)
            label.append(label1 * label2)
            image1.append(combo[0])
            image2.append(combo[1])
    
    # Create the DataFrame
    df = pd.DataFrame(list(zip(image1, image2, labels1,labels2,label)),  columns=['Testing_Signature', 'Original_Signature','Label1','Label2', 'Ouput'])
    df=df[df.Label2!=0]
    df.to_csv("CEDAR_extracted.csv",index=False)

In [43]:
cedar_files_to_csv(55,"png")

In [44]:
df1=pd.read_csv("CEDAR_extracted.csv")

In [45]:
df1.shape

(46860, 5)

In [46]:
df1.shape

(46860, 5)

In [47]:
df1=df1.iloc[:62000,:]


In [48]:
from sklearn.model_selection import train_test_split
#split the df in train, val, test in 80:10:10 and save it as csv files
X_train, X_test, y_train, y_test = train_test_split(df1[['Testing_Signature', 'Original_Signature']], df1['Ouput'], test_size=0.2, random_state=42,
                                                    shuffle=True, stratify=df1['Ouput'])
X_train, X_val, y_train, y_val = train_test_split(df1[['Testing_Signature', 'Original_Signature']], df1['Ouput'], test_size=0.25, random_state=42,
                                                    shuffle=True, stratify=df1['Ouput'])  #0.25*0.8=0.2
train=pd.DataFrame(list(zip(X_train['Testing_Signature'],X_train['Original_Signature'],y_train)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])
val=pd.DataFrame(list(zip(X_val['Testing_Signature'],X_val['Original_Signature'],y_val)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])
test=pd.DataFrame(list(zip(X_test['Testing_Signature'],X_test['Original_Signature'],y_test)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])



In [49]:
train['Ouput'].value_counts()

Ouput
0    23760
1    11385
Name: count, dtype: int64

In [50]:
val['Ouput'].value_counts()

Ouput
0    7920
1    3795
Name: count, dtype: int64

In [51]:
test['Ouput'].value_counts()

Ouput
0    6336
1    3036
Name: count, dtype: int64

In [52]:
train.to_csv("CEDAR_train.csv",index=False)
val.to_csv("CEDAR_val.csv",index=False)
test.to_csv("CEDAR_test.csv",index=False)

In [20]:
from torchvision import transforms
from model import CustomDataset
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()])

# Create an instance of the CustomDataset
dataset = CustomDataset('CEDAR_extracted.csv',transform=transform)


In [21]:
# from model import CustomDataModule
# dm = CustomDataModule('CEDAR_train.csv',transform=transform)
# dm.setup()

# # Get the training data loader
# train_loader = dm.train_dataloader()

# # Iterate over the batches and print the data
# for batch_idx, (data1, data2, label) in enumerate(train_loader):
#     print(f"Batch {batch_idx + 1}:")
#     print("Data 1:")
#     print(data1.shape)
#     print("Data 2:")
#     print(data2.shape)
#     print("Label:")
#     print(label)
#     break

KeyboardInterrupt: 

In [ ]:
import lightning as l
import torch
from model import SiameseNetwork,LightningModel
torch.manual_seed(123)
dm = CustomDataModule('CEDAR_train.csv',batch_size=64,transform=transform)

pytorch_model = SiameseNetwork()

lightning_model = LightningModel(model=pytorch_model, learning_rate=0.0005)

trainer = l.Trainer(
    max_epochs=2,
    accelerator="cpu",
    devices="auto",
    deterministic=True,
)

trainer.fit(model=lightning_model, datamodule=dm)

NameError: name 'CustomDataModule' is not defined

In [ ]:
pytorch_model

In [ ]:
import pandas as pd
df=pd.read_csv("CEDAR_train.csv")

In [ ]:
df=df.iloc[:10000]

In [ ]:
df.shape

(10000, 3)

In [ ]:
df.to_csv("CEDAR_small.csv",index=False)

In [ ]:
def small(file):
    df=pd.read_csv(file)
    print(df.shape)
    # df=df.iloc[:size]
    # df.to_csv(f"{file}_small.csv",index=False)

In [ ]:
small("CEDAR_train.csv")

(46500, 3)


In [ ]:
small("CEDAR_val.csv")

(15500, 3)


In [ ]:
small("CEDAR_test.csv")

(12400, 3)
